In [14]:
import sys
import os
import shutil
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..','..')))
import numpy as np
import cv2
import pandas as pd
from src.preprocessing import preprocessData,thresholding 
from src.TotalCalculation import timeConvertion,SunPosition
from matplotlib import pyplot as plt
from src.ClassPrediction import prediction

In [15]:
image_directory = r'C:\Users\ASUS\Documents\NARIT_internship_data\All_sky_camera_Astropark_Chaingmai\2024-06\2024-06-08'
mask = cv2.imread(r'masks\mask_delete_5.png',cv2.IMREAD_GRAYSCALE)
start_date = '2024-01-01'
end_date = os.path.basename(image_directory)
location = [18.849417,98.9538]
days = timeConvertion().time_duration(start_date,end_date,include_end_date=True).days

LSTM = SunPosition.LSTM(time_zone_offset=7)
EoT = SunPosition.calculate_EoT(day=days)
TC = SunPosition.TimeCorrectionFactor(Longitude=location[1],LSTM=LSTM,EoT=EoT)
dec = SunPosition.declination(day=days)
suntime = SunPosition.DaytimeInfo(latitude=location[0],declination=dec,TC=TC)
sunrise1,sunset1 = SunPosition.DaytimeInfo(latitude=location[0],declination=dec,TC=TC)

In [24]:
def copy_categorical_day(image_directory,output_directory,mask_directory,classes,mode):
    image_directory = image_directory
    mask = cv2.imread(mask_directory,cv2.IMREAD_GRAYSCALE)
    mask = preprocessData().crop_center(mask,crop_size=570)
    images,name = preprocessData().load_images_and_preprocess(path=image_directory,mask=mask,apply_crop_sun=True)
    final,_,_ = thresholding().RBratio(input=images,filename=name,factor=[1,0.83],Time_zone=7,sunrise=sunrise1,sunset=sunset1)
    filtering = lambda x : (x > sunrise1) & (x < sunset1)
    decimal = [timeConvertion().datetime_to_decimal(time=timeConvertion().ticks_to_datetime(ticks=t,time_zone=7)) for t in name]
    if mode == 'day':
        day_indices = [index for index, value in enumerate(decimal) if filtering(value)]
        final_day = final[0:len(day_indices)]
        name_day = [name[i] for i in day_indices]
        percentage = [prediction().CloudRatio(i,mask=mask) for i in final_day]
        classifier = [prediction().classify_sky(i,r) for i,r in percentage]
        for i in range(len(classifier)):
            if classifier[i] == classes:
                original = os.path.join(image_directory,f"{name_day[i]}.png")
                shutil.copy2(original,output_directory)
    if mode == 'night':
        day_indices = [index for index, value in enumerate(decimal) if not filtering(value)]
        final_day = final[0:len(day_indices)]
        name_day = [name[i] for i in day_indices]
        percentage = [prediction().CloudRatio(i,mask=mask) for i in final_day]
        classifier = [prediction().classify_sky(i,r) for i,r in percentage]
        for i in range(len(classifier)):
            if classifier[i] == classes:
                original = os.path.join(image_directory,f"{name_day[i]}.png")
                shutil.copy2(original,output_directory)

In [27]:
image_directory = r'C:\Users\ASUS\Documents\NARIT_internship_data\All_sky_camera_Astropark_Chaingmai\2023-12'
for i in range(len(os.listdir(image_directory))):
    i = i+1
    if i < 10:
        day = str(0)+str(i)
    else:
        day = str(i)
    folder = os.path.join(image_directory,f'2023-12-{day}')
    copy_categorical_day(folder,mask_directory=r'C:\Users\ASUS\Documents\NARIT_internship_2024\NARIT_internship_2024\masks\Domestic observatories\mask_122023.png',
                        output_directory=r'C:\Users\ASUS\Documents\NARIT_internship_data\Test_categorical_New\Test',mode='day',classes='Clear sky (0 oktas)')
    print(f"Day {day} : Copying complete")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\ASUS\\Documents\\NARIT_internship_data\\All_sky_camera_Astropark_Chaingmai\\2023-12\\2024-06-01'

def copy_categorical(target_directory:str,mask_directory:str,mode:str,classes:str,suntime:list):
    mask = cv2.imread(r'C:\Users\ASUS\Documents\NARIT_internship_2024\NARIT_internship_2024\masks\mask_delete_5.png',cv2.IMREAD_GRAYSCALE)
    mask = preprocessData().crop_center(mask,crop_size=570)
    images,name = preprocessData().load_images_and_preprocess(path=r'C:\Users\ASUS\Documents\NARIT_internship_data\All_sky_camera_Astropark_Chaingmai\2024-06\2024-06-08',mask=mask,apply_crop_sun=True)
    final,_,_ = thresholding().RBratio(input=images,filename=name,Time_zone=7,sunrise=sunrise1,sunset=sunset1)
    filtering = lambda x : (x > suntime[0]) & (x < suntime[1])
    decimal = [timeConvertion().datetime_to_decimal(time=timeConvertion().ticks_to_datetime(ticks=t,time_zone=7)) for t in name]
    filename = []
    if mode == 'day':
        classes = []
        day_indices = [index for index, value in enumerate(decimal) if filtering(value)]
        final_day = final[0:len(day_indices)]
        name_day = [name[i] for i in day_indices]
        percentage = [prediction().CloudRatio(i,mask=mask) for i in final_day]
        classifier = [prediction().classify_sky(i,r) for i,r in percentage]
        for i in range(len(classifier)):
            if classifier[i] == classes:
                filename.append(name_day[i])
    if mode == 'night':
        night_indices = [index for index, value in enumerate(decimal) if not filtering(value)]
        final_night = final[0:len(night_indices)]
        name_night = [name[i] for i in night_indices]
        percentage = [prediction().CloudRatio(i,mask=mask) for i in final_night]
        classifier = [prediction().classify_sky(i,r) for i,r in percentage]
        for i in range(len(classifier)):
            if classifier[i] == classes:
                filename.append(name_night[i])
    return filename

name = copy_categorical(target_directory=image_directory,mask_directory=r'C:\Users\ASUS\Documents\NARIT_internship_2024\NARIT_internship_2024\masks\mask_delete_5.png',mode='day',classes='Partly Broken (5 oktas)',suntime=[sunrise1,sunset1])